# Part 1 (Connect from Jupyter to a Neo4j Sandbox): https://dev.neo4j.com/bites_part1
# Repo: https://dev.neo4j.com/bites_repo
# To get started, !pip install py2neo

In [1]:
import pandas as pd
import numpy as np
from py2neo import Graph, Node, Relationship

In [2]:
url = "bolt://44.195.30.251:7687"
pwd = "rag-memorandum-humps"

graph = Graph(url, auth=('neo4j', pwd))

In [3]:
query = "MATCH (n) RETURN COUNT(n)"
result = graph.query(query)
result

COUNT(n)
2642


In [4]:
type(result)

py2neo.cypher.Cursor

In [5]:
query = "MATCH (p:Person {name: 'Eddard Stark'}) RETURN p.name AS name"
result = graph.query(query)
print(result)
print(result.data())

 name         
--------------
 Eddard Stark 

[{'name': 'Eddard Stark'}]


In [6]:
result

(No data)

In [7]:
query = "MATCH (p:Person {name:'Eddard Stark'})-[:INTERACTS_1*1..2]->(p2) RETURN p2.name AS name LIMIT 10"
result = graph.query(query).data()

for record in result:
    print(record['name'])

Marq Piper
Tywin Lannister
Loras Tyrell
Renly Baratheon
Robert Baratheon
Sandor Clegane
Petyr Baelish
Mace Tyrell
Rodrik Cassel
Sansa Stark


In [8]:
type(result)

list

In [9]:
query = """
    MATCH (p1:Person)-[:DEFENDER_COMMANDER]->(b:Battle)<-[:ATTACKER_COMMANDER]-(p2:Person)
    RETURN p2.name AS attacker, b.name AS battle, p1.name AS defender LIMIT 10
"""

result_df = graph.query(query).to_data_frame()
result_df.head(10)

,attacker,battle,defender
0,Jaime Lannister,Battle of the Golden Tooth,Vance
1,Jaime Lannister,Battle of the Golden Tooth,Clement Piper
2,Gregor Clegane,Battle at the Mummer's Ford,Beric Dondarrion
3,Andros Brax,Battle of Riverrun,Tytos Blackwood
4,Jaime Lannister,Battle of Riverrun,Tytos Blackwood
5,Andros Brax,Battle of Riverrun,Edmure Tully
6,Jaime Lannister,Battle of Riverrun,Edmure Tully
7,Harrion Karstark,Battle of the Green Fork,Tywin Lannister
8,Medger Cerwyn,Battle of the Green Fork,Tywin Lannister
9,Roose Bolton,Battle of the Green Fork,Tywin Lannister


In [10]:
query = """
    MATCH (p1:Person)-[:DEFENDER_COMMANDER]->(b:Battle)<-[:ATTACKER_COMMANDER]-(p2:Person)
    RETURN p2.name AS attacker, b.name AS battle, p1.name AS defender LIMIT 10
"""

result_arr = graph.query(query).to_ndarray()
result_arr

array([['Jaime Lannister', 'Battle of the Golden Tooth', 'Vance'],
       ['Jaime Lannister', 'Battle of the Golden Tooth', 'Clement Piper'],
       ['Gregor Clegane', "Battle at the Mummer's Ford",
        'Beric Dondarrion'],
       ['Andros Brax', 'Battle of Riverrun', 'Tytos Blackwood'],
       ['Jaime Lannister', 'Battle of Riverrun', 'Tytos Blackwood'],
       ['Andros Brax', 'Battle of Riverrun', 'Edmure Tully'],
       ['Jaime Lannister', 'Battle of Riverrun', 'Edmure Tully'],
       ['Harrion Karstark', 'Battle of the Green Fork',
        'Tywin Lannister'],
       ['Medger Cerwyn', 'Battle of the Green Fork', 'Tywin Lannister'],
       ['Roose Bolton', 'Battle of the Green Fork', 'Tywin Lannister']],
      dtype='<U27')

In [11]:
a = Node("Person", name="Clair")
b = Node("Person", name="Emil")
ab = Relationship(a, "KNOWS", b)
graph.create(ab)

In [14]:
graph.exists(ab)

True